In [ ]:
import pandas as pd
import numpy as np
import random

# 포지션 및 영웅 설정
POSITIONS = ["탱커", "딜러", "힐러"]
HEROES = {
    "탱커": ["라인하르트", "자리야", "윈스턴"],
    "딜러": ["겐지", "한조", "소전"],
    "힐러": ["메르시", "루시우", "아나"]
}

# 원하는 닉네임 목록 (순서와 중복은 사용자의 취향에 맞게 조정)
NICKNAMES = [
    "hyeon", "어쩔", "아빠는새우튀김", "비폭력주의자", "월화", 
    "몽수빈", "둘트", "hock", "여쪽이", "식동이"
]

# 스탯이 매우 안 좋은 닉네임 목록
POOR_NAMES = {"몽수빈", "여쪽이", "식동이"}

def generate_row(game_id):
    # 플레이어 이름은 NICKNAMES에서 무작위 선택
    chosen_name = random.choice(NICKNAMES)
    
    # 포지션 및 영웅 선정
    pos = random.choice(POSITIONS)
    hero = random.choice(HEROES[pos])
    team = random.choice(["A", "B"])
    map_type = random.choice(["쟁탈", "밀기", "호위", "격돌", "혼합"])
    
    # 플레이어 스탯 생성 (선택된 닉네임에 따라 다르게)
    if chosen_name in POOR_NAMES:
        # 매우 안 좋은 스탯: 낮은 처치, 낮은 도움, 높은 죽음, 낮은 피해량, (힐러/탱커의 경우) 낮은 치유/경감
        stats = {
            "처치": np.random.poisson(1),
            "도움": np.random.poisson(1),
            "죽음": np.random.poisson(10),
            "피해": np.random.randint(1000, 5000),
            "치유": np.random.randint(0, 2000) if pos == "힐러" else 0,
            "경감": np.random.randint(0, 2000) if pos == "탱커" else 0
        }
    else:
        # 기본 스탯 생성
        stats = {
            "처치": np.random.poisson(10),
            "도움": np.random.poisson(5),
            "죽음": np.random.poisson(6),
            "피해": np.random.randint(3000, 20000),
            "치유": np.random.randint(0, 16000) if pos == "힐러" else 0,
            "경감": np.random.randint(0, 15000) if pos == "탱커" else 0
        }
        
    # soft_label 계산: 스탯에 기반하여 0과 1 사이로 클리핑 (여기서 poor stats인 경우 자연히 낮은 값이 나옵니다)
    soft_label = round(np.clip(0.3 + (stats["처치"] - stats["죽음"]) * 0.02 + np.random.normal(0, 0.05), 0, 1), 3)
    
    return {
        "이름": chosen_name,
        "포지션": pos,
        "영웅": hero,
        "게임번호": game_id,
        "팀": team,
        "전장": map_type,
        "밀기거리": np.random.randint(0, 130),
        **stats,
        "soft_label": soft_label,
        "A팀점수": np.random.randint(0, 4),
        "B팀점수": np.random.randint(0, 4),
        "결과": random.choice(["승", "패", "무"])
    }

# 데이터 생성 (예: 1000게임)
rows = [generate_row(i) for i in range(1000)]
df = pd.DataFrame(rows)

# 엑셀 파일로 저장 (원하는 파일명으로)
df.to_excel("ow_player_logs_softlabel_0to1.xlsx", index=False)
print("데이터 생성 완료")